# SAS 프로그램 결과

In [63]:
import saspy #SAS출력 코드

winlocal 입력

## 19장

In [64]:
%%SAS

LIBNAME ex 'C:\Biostat';
RUN;

/*19장 연습문제 불러오기*/
PROC IMPORT DBMS=sav
DATAFILE='C:\Biostat\Exam 19.1A.sav'
OUT=ex.ex19_1a REPLACE;
RUN;

%macro chap19(name=,no=);
%do i=12 %to &no.;
	PROC IMPORT DBMS=sav
		DATAFILE="C:\Biostat\Exam 19.&i."
		OUT=ex.&name.&i. REPLACE;
	RUN;
%end;
%mend;

%chap19(name=ex19_,no=18);


### EXAMPLE 19.1

In [65]:
%%SAS

PROC CORR DATA=ex.ex19_1A PEARSON;
     VAR WingLeingth TailLength;
RUN;

### EXAMPLE 19.2

In [66]:
%%SAS

PROC IML;
	r= 0.870;
	n = 12;
	z = 0.5 * log((1 + r) / (1 - r));
	a = 0.5 * log((1 + 0.750) / (1 - 0.750));
	cz = round((z - a) / (sqrt(1 / (n - 3))), .0001);
	p = round((1 - probnorm(cz)) * 2, .01);
	PRINT cz p;
RUN;

ods graphics off;ods exclude all;ods noresults;
PROC CORR DATA=ex.ex19_1A nosimple fisher(rho0=0.750 biasadj=no);
    VAR WingLeingth TailLength;
    ODS OUTPUT FisherPearsonCorr=fisher;
RUN;
ods graphics on;ods exclude none;ods results;

PROC PRINT DATA=fisher;
RUN;

### EXAMPLE 19.3

In [67]:
%%SAS

proc iml;
	reset nolong;
	r=0.870;f=3.72;
	L1=((1+f)*r+(1-f))/((1+f)+(1-f)*r);
	L2=((1+f)*r-(1-f))/((1+f)-(1-f)*r);
	print L1 L2;
run;

L1,L2
0.5890551,0.96331


### EXAMPLE 19.4

In [68]:
%%SAS

PROC POWER;
    ONECORR DIST=fisherz
    NULLCORR=0
    CORR=0.870
    NTOTAL=12
    POWER=.;
RUN;

### EXAMPLE 19.5a

In [69]:
%%SAS

PROC POWER;
    ONECORR DIST=fisherz
    NULLCORR=0
    CORR=0.5
    NTOTAL=.
    POWER=0.99;
RUN;

### EXAMPLE 19.5b

In [70]:
%%SAS

PROC IML;	
    RESET nolog;
	r=0.870;f=2.98;
	L1=((1+f)*r+(1-f))/((1+f)+(1-f)*r);
	L2=((1+f)*r-(1-f))/((1+f)-(1-f)*r);
	diff=L2-L1;
    PRINT L1 L2 diff;
RUN;
QUIT;

L1,L2,diff
0.6567733,0.9544068,0.2976335


### EXAMPLE 19.6

In [71]:
%%SAS

PROC IML;	
    RESET nolog;
	r1=0.78;r2=0.84;
	z1=1.0454;z2=1.2212;
	n1=98;n2=95;
	
	z=(z1-z2)/sqrt((1/(n1-3))+(1/(n2-3)));
	
	p_value=probnorm(z)*2;
	zw=((n1-3)*z1+(n2-3)*z2)/(n1-3+n2-3);
	rw=(exp(2*(zw))-1)/(exp(2*(zw))+1);
    PRINT z p_value zw rw;
RUN;
QUIT;

z,p_value,zw,rw
-1.20186,0.2294179,1.1318898,0.8116651


### EXAMPLE 19.7

In [72]:
%%SAS

PROC IML;	
    RESET nolog;
	r1=0.78;r2=0.84;
	z1=1.0454;z2=1.2212;
	n1=98;n2=95;
	
	z_beta=(abs(z1-z2))/sqrt((1/(n1-3))+(1/(n2-3)))-1.96;
	beta=1-probnorm(z_beta);
	power=1-beta;
    PRINT beta power;
RUN;
QUIT;

beta,power
0.7758165,0.2241835


### EXAMPLE 19.8

In [73]:
%%SAS

PROC IML;	
    RESET nolog;
	z1=-probit(0.05/2);z2=-probit(0.10);
	diff=0.5;
	n=2*(((z1+z2)/diff)**2)+3;
    PRINT n;
RUN;
QUIT;

n
87.059384


### EXAMPLE 19.9

In [74]:
%%SAS

PROC IML;	
    RESET nolog;
	n={24,29,32};
	r={0.52,0.56,0.87};
    z=0.5*log((1+r)/(1-r));

	z2=z#z;
	n_z2=(n-3)#z2;
	n_z=(n-3)#z;
	chi=sum(n_z2)-(sum((n_z))**2/sum(n-3));
	p=1-probchi(chi,2);

	zw=(sum((n_z))/sum(n-3));
	rw=(exp(2*(zw))-1)/(exp(2*(zw))+1);
    PRINT chi p zw rw;
RUN;
QUIT;

chi,p,zw,rw
9.4764447,0.0087542,0.8844225,0.7086279


### EXAMPLE 19.10

In [75]:
%%SAS

PROC IML; 
	n1 = 24 ;   n2 = 29;    n3 = 32;
	r1 = 0.52; r2 = 0.56;  r3 = 0.87;

	rw = 0.71 ; 

	xp1 = round((n1 * (r1 - rw) ** 2) / (1 - r1 * rw) ** 2, .0001) ; 
	xp2 = round((n2 * (r2 - rw) ** 2) / (1 - r2 * rw) ** 2, .0001) ; 
	xp3 = round((n3 * (r3 - rw) ** 2) / (1 - r3 * rw) ** 2, .0001) ; 

	xp = xp1 + xp2 + xp3 ;
	p = round(1- probchi(xp, 2), .0001);

	PRINT xp1 xp2 xp3 p ; 

RUN; 

xp1,xp2,xp3,p
2.1774,1.7981,5.6051,0.0083


### EXAMPLE 19.11

In [76]:
%%SAS

PROC IML;
	%macro ex19_11(x1,x2);
	n={24,29,32}; 
	r={0.52,0.56,0.87}; 
	z=0.5*log((1+r)/(1-r));
	diff=round(z[&x1]-z[&x2],0.0001);
	se=round(sqrt((1/2)*((1/(n[&x1]-3))+(1/(n[&x2]-3)))),0.001) ; 
	q=round(diff/se,0.001); 
		if q>=3.314 then conclusion='Reject H0' ; 
		else if q <3.314 then conclusion='Do not reject H0' ; 
	PRINT diff se q conclusion; 
%mend ; 
%ex19_11 (3,1) ; 
%ex19_11 (3,2) ; 
%ex19_11 (2,1) ; 
QUIT;

diff,se,q,conclusion
0.7567,0.203,3.728,Reject H0
diff,se,q,conclusion
0.7002,0.191,3.666,Reject H0
diff,se,q,conclusion
0.0565,0.207,0.273,Do not reject H0


### EXAMPLE 19.12

In [77]:
%%SAS

PROC CORR DATA= ex.ex19_12 spearman nosimple;
	VAR x y ;
RUN;

### EXAMPLE 19.13

In [78]:
%%SAS

PROC CORR DATA= ex.ex19_13 spearman nosimple;
	VAR x y ;
RUN;

### EXAMPLE 19.14

In [79]:
%%SAS

DATA ex19_14;
INPUT species1 species2 @@; 
   CARDS;
2.718 2.718
1.718 1.718 
1.218 1.218
0.885 0.268
0.635 0.125
0.435 0.435
0.268 0.635
0.125 0.885
;
RUN;

PROC CORR DATA= ex19_14 pearson nosimple;
	VAR species1 species2 ;
RUN;

### EXAMPLE 19.15

In [80]:
%%SAS

ods graphics off; ods exclude all; ods noresults;
PROC FREQ DATA = ex.ex19_15;
	TABLES Disease*Insect / norow nocol nopercent expected chisq out=table19_15;
	TEST gamma;
	ODS OUTPUT ChiSq=ChiSq Gamma=Gamma;
RUN;
ods graphics on; ods exclude none; ods results;

PROC PRINT DATA=ChiSq label;
    WHERE Statistic="파이 계수";
    VAR Statistic Value;
RUN;

PROC PRINT DATA=Gamma label;
    VAR Label1 cValue1;
RUN;

PROC IML;	
    RESET nolog;
	f11=6; f12=4;
	f21=0; f22=4;

	r_n=round(((f11+f22)-(f12+f21))/((f11+f22)+(f12+f21)), 0.01);

	PRINT r_n;
RUN;
QUIT;

### EXAMPLE 19.16

In [81]:
%%SAS


%macro biserial(version, data= ,contin= ,binary= ,out=);

%if &version ne %then %put BISERIAL macro Version 2.2;

options nonotes;
* exclude observations with missing variables *;
data &out;
 set &data;
 where &contin>.;
 if &binary>.;
 run;

* compute the ranks for the continuous variable *;
proc rank data=&out out=&out ;
 var &contin;
 ranks r_contin;
 run;

* compute proportion of binary, std of contin, and n *;
proc means data=&out noprint;
 var &binary &contin;
 output out=_temp_(keep=p stdy n) mean=p std=stdx stdy n=n;
 run;

* sort by the binary variable *;
proc sort data=&out;
 by descending &binary;
 run;

* compute mean of contin and rank of contin var *;
proc means data=&out noprint;
 by notsorted &binary;
 var &contin r_contin;
 output out=&out mean=my r_contin;
 run;

* restructure the means computed in the step above *;
proc transpose data=&out out=&out(rename=(col1=my1 col2=my0));
 var r_contin my;
 run;

* combine the data needed to compute biserial correlation *;
data &out;
 set &out(drop= _name_ _label_);
 retain r1 r0 ;
 if _n_=1 then do;
  r1=my1;
  r0=my0;
 end;
 else do;
  set _temp_;
  output;
 end;
 run;

* compute point biserial correlation *;
proc corr data=&data  noprint outp=_temp_;
 var &binary &contin;
 run;



* extract the point biserial correlation from the matrix *;
data _temp_(keep=pntbisrl);
 set _temp_(rename=(&contin=pntbisrl));
 if _TYPE_='CORR' and &binary<>1 then output;

 run;

options notes;
* compute biserial and rank biserial *;
data &out;
 merge _temp_  &out;
 if pntbisrl=1 then delete;
 h=probit(1-p);
 u=exp(-h*h/2)/sqrt(2*arcos(-1));
 biserial=p*(1-p)*(my1-my0)/stdy/u;
 rnkbisrl=2*(r1-r0)/n;

 keep biserial pntbisrl rnkbisrl;
 label biserial='Biserial Corr'
       pntbisrl='Point Biserial Corr'
       rnkbisrl='Rank Biserial Corr';
 run;

%mend;

%biserial(data=ex.ex19_16, contin=Y, binary=X, out=out19_16);

PROC PRINT DATA=out19_16 label;
	VAR pntbisrl;
RUN;

PROC IML;	
	RESET nolog;
	n=13;
	r_pb=0.59830;
	r2_pb=(r_pb)**2;
	t=round((r_pb/sqrt((1-r2_pb)/(n-2))), 0.001);
	p_value = round((1 - probt(t, n-2))*2, 0.001);
	PRINT t p_value;
RUN;
QUIT;

OBS,Point Biserial Corr
1,0.59830
t,p_value
2.476,0.031


### EXAMPLE 19.17

In [82]:
%%SAS

PROC IML;	
    RESET nolog;
	sst=220.17; ssb=198.31;sse=21.86;
	df_b=6;df_e=7;
	msb=ssb/df_b;
	mse=sse/df_e;

	r=((msb-mse)/(msb+mse));
	f=msb/mse;

	p=1-probf(f,df_b,df_e);

	PRINT r f p;
RUN;
QUIT;

r,f,p
0.8273449,10.583791,0.003251


### EXAMPLE 19.18

In [83]:
%%SAS

PROC MEANS DATA=ex.ex19_18 noprint;
	VAR A B;
	OUTPUT OUT=dice_baselinemin min=var1 var2;
RUN;

proc means data=ex.ex19_18 noprint;
	VAR A B;
	OUTPUT OUT=dice_baselinemax max=var1 var2;
RUN;

DATA dice_baselineall;
	SET dice_baselinemin dice_baselinemax;
	DROP _type_ _freq_;
	IF _N_=1 THEN var1=floor(min(var1,var2));
	IF _N_=1 THEN var2=floor(min(var1,var2));
	IF _N_=2 THEN var1=ceil(max(var1,var2));	
	IF _N_=2 THEN var2=ceil(max(var1,var2));
RUN;

DATA dice_baselineall;
	SET ex.ex19_18 dice_baselineall;
RUN;

PROC GPLOT UNIFORM DATA=dice_baselineall;
plot A*B var1*var2/overlay vaxis=axis1 haxis=axis2 nolegend frame;
axis1 label=(a=90 'Cortisol Every Hour') minor=none;
axis2 label=('Cortisol Every Two Hours') minor=none;
symbol1 value=star color=black interpol=none;
symbol2 value=none color=black interpol=join;
title "Concordance Correlation Coefficient";
RUN;


PROC IML;
*******************************************************************************
  *  Enter the appropriate SAS data set name in the use statement and enter the *
  *  appropriate variable names in the read statements.                         *
  *******************************************************************************;
use ex.ex19_18;
read all var {A} into var1;
read all var {B} into var2;
*******************************************************************************
*  The IML module, labeled concorr, starts next.                              *
*******************************************************************************;
start concorr;
nonmiss=loc(var1#var2^=.);
var1=var1[nonmiss];
var2=var2[nonmiss];
free nonmiss;
n=nrow(var1);
mu1=sum(var1)/n;
mu1=round(mu1,0.0001);
mu2=sum(var2)/n;
mu2=round(mu2,0.0001);
sigma11=ssq(var1-mu1)/(n-1);
sigma11=round(sigma11,0.0001);
sigma22=ssq(var2-mu2)/(n-1);
sigma22=round(sigma22,0.0001);
sigma12=sum((var1-mu1)#(var2-mu2))/(n-1);
sigma12=round(sigma12,0.0001);
lshift=(mu1-mu2)/((sigma11#sigma22)##0.25);
rho=sigma12/sqrt(sigma11#sigma22);
rho=round(rho,0.0001);
z=log((1+rho)/(1-rho))/2;
se_z=1/sqrt(n-3);
t=tinv(0.975,n-3);
z_low=z-(se_z#t);
z_upp=z+(se_z#t);
rho_low=(exp(2#z_low)-1)/(exp(2#z_low)+1);
rho_low=round(rho_low,0.0001);
rho_upp=(exp(2#z_upp)-1)/(exp(2#z_upp)+1);
rho_upp=round(rho_upp,0.0001);
crho=(2#sigma12)/((sigma11+sigma22)+((mu1-mu2)##2));
crho=round(crho,0.0001);
z=log((1+crho)/(1-crho))/2;
if sigma12^=0 then do;
   t1=((1-(rho##2))#(crho##2))/((1-(crho##2))#(rho##2));
   t2=(2#(crho##3)#(1-crho)#(lshift##2))/(rho#((1-(crho##2))##2));
   t3=((crho##4)#(lshift##4))/(2#(rho##2)#((1-(crho##2))##2));
   se_z=sqrt((t1+t2-t3)/(n-2));
end;
else se_z=sqrt(2#sigma11#sigma22)/((sigma11+sigma22+((mu1-mu2)##2))#(n-2));
t=tinv(0.975,n-2);
z_low=z-(se_z#t);
z_upp=z+(se_z#t);
crho_low=(exp(2#z_low)-1)/(exp(2#z_low)+1);
crho_low=round(crho_low,0.0001);
crho_upp=(exp(2#z_upp)-1)/(exp(2#z_upp)+1);
crho_upp=round(crho_upp,0.0001);
Results=n//mu1//mu2//sigma11//sigma22//sigma12//rho_low//rho//rho_upp//
        crho_low//crho//crho_upp;
r_name={'SampleSize' 'Mean_1' 'Mean_2' 'Variance_1' 'Variance_2' 'Covariance' 'Corr LowerCL'
       'Corr' 'Corr UpperCL' 'ConcCorr LowerCL' 'ConcCorr' 'ConcCorr UpperCL'};
print 'The Estimated Correlation and Concordance Correlation (and 95% Confidence Limits)';
print Results [rowname=r_name];
finish concorr;
*******************************************************************************
*  The IML module, labeled concorr, is finished.                              *
*******************************************************************************;
RUN concorr;